In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Research ISB - Data Science for Product Managers/kidsInMindSubtitles2004.csv")

In [ ]:
df.rename(columns = {'subtitles':'text', 'Language':'labels'}, inplace = True)
df=df.head(1000)
df.labels=df.labels.astype(float)

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import Dataset,load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
dataset = Dataset.from_pandas(df,preserve_index=False) 
dataset = dataset.train_test_split(test_size=0.3) 

dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Unnamed: 0.1', 'Movie_Title', 'Sex_Nudity', 'Violence_Gore', 'labels', 'Movie_Description', 'Sex_Nudity_Description', 'Violence_Gore_Description', 'Language_Description', 'Substance_Use_Description', 'Discussion_Topics_Description', 'Message', 'HyperLink', 'text'],
        num_rows: 700
    })
    test: Dataset({
        features: ['Unnamed: 0', 'Unnamed: 0.1', 'Movie_Title', 'Sex_Nudity', 'Violence_Gore', 'labels', 'Movie_Description', 'Sex_Nudity_Description', 'Violence_Gore_Description', 'Language_Description', 'Substance_Use_Description', 'Discussion_Topics_Description', 'Message', 'HyperLink', 'text'],
        num_rows: 300
    })
})

In [ ]:
dataset['train']['text'][:5]

['Scared the living crap out of me.Meryl Streep she wasn\'t.Who\'s Meryl Streep?Zoe, she\'s like Katie Holmes\nto our parents.So, what\'s next, ladies?-I have to go home.\n-No.I can\'t have another fight with\nmy parents. They think I\'m in bed.Were you kids in bed early?-I stayed up till 10.\n-After 9, Rhett was asleep.-Who\'s driving them?\n-I will.Nicki\'s selling coffee at school,\nI\'m helping her.It sucks.Do not say "sucks," say "disappoints."I don\'t need a lunch.\nI\'ll go with Tash.Eat pizza three days in a row?\nI think not.That\'s really good.The proportions are perfect......but do you draw\nwhat you really see?The simplest is the hardest.\nSeeing what\'s in front of you.What\'s on for tonight?I am so grounded. I got busted\nsneaking back inside last night.-Damn, how\'d you manage that?\n-My dad set the motion detector.Our parents are getting\nparanoid in their old age.Mine still trust me.It\'s beautiful.Not a bad couple of decades.Better than the first two\nwithout you.To 2

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.26.1",
  "vocab_size": 30522
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapsh

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=1)


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.26.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/pytorch_model.bin
Some we

In [ ]:
model.resize_token_embeddings(len(tokenizer))

Embedding(30522, 768, padding_idx=0)

In [ ]:
from datasets import load_metric
from sklearn.metrics import mean_squared_error

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  num_train_epochs=3,
                                  save_total_limit = 2,
                                  save_strategy = 'no',
                                  load_best_model_at_end=False
                                  )


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics
)
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Sex_Nudity_Description, Violence_Gore, Substance_Use_Description, Sex_Nudity, HyperLink, Unnamed: 0.1, Discussion_Topics_Description, Unnamed: 0, Movie_Title, Movie_Description, Violence_Gore_Description, text, Message, Language_Description. If Sex_Nudity_Description, Violence_Gore, Substance_Use_Description, Sex_Nudity, HyperLink, Unnamed: 0.1, Discussion_Topics_Description, Unnamed: 0, Movie_Title, Movie_Description, Violence_Gore_Description, text, Message, Language_Description are not expected by `DistilBertForSequenceClassification.forward`,  

Epoch,Training Loss,Validation Loss,Rmse
1,11.779600,7.900503,2.810784
2,7.658600,7.474644,2.733979
3,6.080400,6.918003,2.630210


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Sex_Nudity_Description, Violence_Gore, Substance_Use_Description, Sex_Nudity, HyperLink, Unnamed: 0.1, Discussion_Topics_Description, Unnamed: 0, Movie_Title, Movie_Description, Violence_Gore_Description, text, Message, Language_Description. If Sex_Nudity_Description, Violence_Gore, Substance_Use_Description, Sex_Nudity, HyperLink, Unnamed: 0.1, Discussion_Topics_Description, Unnamed: 0, Movie_Title, Movie_Description, Violence_Gore_Description, text, Message, Language_Description are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Sex_Nudity_Description, Violence_Gor

TrainOutput(global_step=132, training_loss=8.506208477598248, metrics={'train_runtime': 7933.8924, 'train_samples_per_second': 0.265, 'train_steps_per_second': 0.017, 'total_flos': 278176576204800.0, 'train_loss': 8.506208477598248, 'epoch': 3.0})

In [ ]:
# save the model/tokenizer

model.save_pretrained("model")
tokenizer.save_pretrained("tokenizer")



# load the model/tokenizer

from transformers import AutoModelForTokenClassification
model = AutoModelForSequenceClassification.from_pretrained("model")
tokenizer = AutoTokenizer.from_pretrained("tokenizer")

Configuration saved in model/config.json
Model weights saved in model/pytorch_model.bin
tokenizer config file saved in tokenizer/tokenizer_config.json
Special tokens file saved in tokenizer/special_tokens_map.json
loading configuration file model/config.json
Model config DistilBertConfig {
  "_name_or_path": "model",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "regression",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "vocab_size": 30522
}

loading weights file model/pytorch_model.bin
All mod

In [ ]:
ds=pd.read_csv("/content/drive/MyDrive/Research ISB - Data Science for Product Managers/kidsInMindSubtitles2004.csv")

In [ ]:
from transformers import Trainer
trainer = Trainer(model=model)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True) 

def pipeline_prediction(text):
    df=pd.DataFrame({'text':[text]})
    dataset = Dataset.from_pandas(df,preserve_index=False) 
    tokenized_datasets = dataset.map(tokenize_function)
    raw_pred, _, _ = trainer.predict(tokenized_datasets) 
    return(raw_pred[0][0])

pipeline_prediction(ds['subtitles'][425])

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1
  Batch size = 8


5.540816

In [ ]:
pipeline_prediction(ds['subtitles'][700])

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1
  Batch size = 8


3.392324

In [ ]:
pipeline_prediction(ds['subtitles'][200])

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1
  Batch size = 8


5.2081404

In [ ]:
ds['Language'][200]

4

In [ ]:
ds['Language'][700]

1

In [ ]:
ds['Language'][425]

2

In [ ]:
pipeline_prediction(ds['subtitles'][989])

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1
  Batch size = 8


4.914034

In [ ]:
ds['Language'][989]

3